# Co-factor swap

<!-- The balance of co-factors within the cells is important 
In the implementated pathway of progesterone nine NADP and one NAD molecules 
In order for the cell to produce one pregesterone molecule through the implementated pathway nine NADPH molecules and one NAD $^+$ molecules are needed. 
 -->



The balance of co-factors within a cell is important to obtain a high theoretical yield of a given product (King, Zachary A., and Adam M. Feist, 2014). The NADP(H) and NAD(H) co-factors are needed in order for a cell to produce progesterone through the implemented pathway. Therefore, by increasing the concentration of available NADP(H) and NAD(H) co-factors in the cell, it should be theoretically possible to increase the yield of progesterone. 
This can be done using the “CofactorSwapOptimization” algorithm where possible target reactions to swap co-factors can be found.
The obvious choice would be to increase available NAD(H) and NADP(H) on the cost of FAD(H2). However, no FADH2 in the cytosol is included in the model which makes this approach impossible. Therefore, we instead used the algorithm to increase the concentration of available NADP(H) on the cost of NAD(H). Since nine NADP(H) and only one NAD(H) are used in the implemented pathway, this approach should also theoretically increase the yield of progesterone.

In the code the following was done:
- Model iMM904_progesterone was loaded
- A demand for progesterone was made 
- The objective was set to maximize the production of progesterone
- The theoretical yield was set to production of progesterone divided by the optake of glucose
- The algorithm was run and the result was visualized

In [455]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield, biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo import load_model

In [456]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [457]:
# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 


In [458]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c


In [459]:
# Setting the progesterone yield as the formation of progesterone divided by the optake of glucose
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)
py

In [460]:
# Running the CofactorSwapOptimization Algorithm
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors swapped.
swap = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nadp_c","nadph_c"],["nad_c","nadh_c"]),objective_function=py)
print(f'Co-factor swap optimization of the model {model.id} used to increase available {swap._metadata["swap_pairs"][0][0]} and {swap._metadata["swap_pairs"][0][1]} on the cost of {swap._metadata["swap_pairs"][1][0]} and {swap._metadata["swap_pairs"][1][1]}')
solution_1 = swap.run(max_size = 10)

Co-factor swap optimization of the model iMM904_progesterone used to increase available nadp_c and nadph_c on the cost of nad_c and nadh_c
Starting optimization at Wed, 23 Nov 2022 15:00:13


HBox()

Finished after 00:03:52


In [461]:
# Printing results
solution_1

,index,targets,fitness
0,0,"(GAPD,)",0.323077
1,1,"(ALCD2x_copy1,)",0.323077
2,2,"(MTHFD, MTHFD2i)",0.323077
3,4,"(MDH, HMGCOAR)",0.323077
4,8,"(ALCD2x_copy2,)",0.323077
5,13,"(MDH, C3STDH2, R01456)",0.262500
6,14,"(MDH, C3STDH2, C3STKR1)",0.262500
7,15,"(HMGCOAR,)",0.253012
8,17,"(MDH,)",0.253012
9,23,"(SQLS, ECYP11A1, C3STKR2, C3STDH2)",0.240000


The reactions listed above can be used as targets for co-factor swapping to increase the theoretical yield of progesterone. The reaction with the highest fitness is GAPD which corospond with finding by King, Zachary A. and Adam M. Feist. This gene was therefore investigated further: 

In [462]:
# Stats of the GAPD reaction
model.reactions.GAPD

Reaction identifier,GAPD
Name,Glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x07f5c219fd3a0
Stoichiometry,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate <=> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YJL052W or YJR009C or YGR192C
Lower bound,-999999.0
Upper bound,999999.0


NAP(+) is converted into NADH in the GAPD reaction, however, by swapping the co-factor into NADP(H) it should be possible to get a higher concentration of available NADPH in the cell and thereby increasing the yield of progesterone. As seen above three genes in the model are able to make the GAPD. Thus, to be able to make co-factor swapping these genes have to be knocked out and a NADP(+) dependent GAPD needs to be implemented in the model. This is done in the following code: 

In [463]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [464]:
# Knock out of GAPD genes
KOs = ["YJL052W","YJR009C","YGR192C"]
model_coswap = model.copy()
model_coswap.id = "iMM904_progesterone_coswap"
print(f"Model {model_coswap.id} were made as a copy of {model.id}")
print(f"For model {model_coswap.id}:")
for KO in KOs:
    model_coswap.genes.get_by_id(KO).knock_out()
    print(f"{KO} were knocked out")

Model iMM904_progesterone_coswap were made as a copy of iMM904_progesterone
For model iMM904_progesterone_coswap:
YJL052W were knocked out
YJR009C were knocked out
YGR192C were knocked out


In [465]:
# Adding NADP dependent GAPD reaction
print(f"For model {model_coswap.id}:")
with open("data/co-factor_swap_reactions.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        r = Reaction(line[0])
        r.name = line[2]
        if line[3] != "":
            r.subsystem = line[3]
        r.lower_bound = float(line[4])
        r.upper_bound = float(line[5])
        if line[1] != "":
            r.gpr = GPR.from_string(line[1])
        for i in range(int(len(line[6:])/2)):
            metaboliteID = line[6+i*2]
            if metaboliteID != "":
                metaboliteID = model_coswap.metabolites.get_by_id(metaboliteID)
                bin = float(line[7+i*2])
                r.add_metabolites({metaboliteID:bin})
        print(f'Adding reaction {r} | enzyme {r.gpr}')
        model_coswap.add_reactions([r])

For model iMM904_progesterone_coswap:
Adding reaction R01063: g3p_c + nadp_c + pi_c <=> 13dpg_c + h_c + nadph_c | enzyme GDP1


In [466]:
# Add demand reaction for progesterone
models = [model,model_coswap]
for model in models:
    model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
    print(f'Model {model.id}: Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Model iMM904_progesterone: Adding reaction progesterone demand: progesterone_c --> 
Model iMM904_progesterone_coswap: Adding reaction progesterone demand: progesterone_c --> 


In [467]:
# Calculate µ
biomass = []
for i, model in enumerate(models):
    biomass.append(model.optimize().objective_value)
    print(f'{model.id}: µ = {biomass[i]}')


iMM904_progesterone: µ = 0.2878657037040161
iMM904_progesterone_coswap: µ = 0.29373393760455596


In [468]:
# Calculate flux towards progesterone
pp = []
for model in models:
    with model:
        objective_reaction = model.reactions.R02216
        model.objective = objective_reaction
        pp.append(model.optimize())

print(f'{model.id}: progesterone flux = {pp[0].objective_value}')
print(f'{model_coswap.id}_coswap: progesterone flux = {pp[1].objective_value}')

iMM904_progesterone_coswap: progesterone flux = 0.14285714285714285
iMM904_progesterone_coswap_coswap: progesterone flux = 0.14285714285714285


In [469]:
# Calculate procentage increase in µ
print(f"Procentage increase in µ: {round((biomass[1]/biomass[0]-1)*100,2)}%")

Procentage increase in µ: 2.04%


From the results it is clear that the flux towards progesterone does not change by the co-factor swapping, however, the growth rate does.

In [470]:
# Saving new model
outfilename = "models/iMM904_progesterone_coswap.xml"
print(f"Saving to {outfilename}")
write_sbml_model(model_coswap, outfilename)

Saving to models/iMM904_progesterone_coswap.xml


### References
King, Zachary A., and Adam M. Feist. "Optimal cofactor swapping can increase the theoretical yield for chemical production in Escherichia coli and Saccharomyces cerevisiae." Metabolic Engineering 24 (2014): 117-128.

# RESTEN ER BARE JUNK

In [471]:
# Printing result
print("The following reactions can be used as targets for co-factor swapping:\n")
target_set = set()
for target in solution_1.data_frame.targets:
    for id in target:
        target_set.add(id)
for id in target_set:
    print(model.reactions.get_by_id(id))

The following reactions can be used as targets for co-factor swapping:

C4STMO2: 4mzym_c + 3.0 h_c + 3.0 nadph_c + 3.0 o2_c --> 4.0 h2o_c + 3.0 nadp_c + zym_int1_c
ALCD2x_copy1: etoh_c + nad_c --> acald_c + h_c + nadh_c
C14STR: 44mctr_c + h_c + nadph_c --> 44mzym_c + nadp_c
R01456: dehydrocholesterol_c + h_c + nadph_c --> cholesterol_c + nadp_c
C3STKR1: 4mzym_int2_c + h_c + nadph_c --> 4mzym_c + nadp_c
ALCD2x_copy2: etoh_c + nad_c <=> acald_c + h_c + nadh_c
HMGCOAR: coa_c + mev__R_c + 2.0 nadp_c <=> 2.0 h_c + hmgcoa_c + 2.0 nadph_c
R07215: h_c + lathosterol_c + nadph_c + o2_c --> dehydrocholesterol_c + 2.0 h2o_c + nadp_c
ECYP11A1: cholesterol_c + 6.0 h_c + 6.0 nadph_c + 3.0 o2_c --> 4.0 h2o_c + methylpentanal_c + 6.0 nadp_c + pregnenolone_c
C3STDH1: 4mzym_int1_c + nad_c --> 4mzym_int2_c + co2_c + h_c + nadh_c
R05703: cholesta724dien3betaol_c + h_c + nadph_c <=> lathosterol_c + nadp_c
C3STKR2: h_c + nadph_c + zym_int2_c --> nadp_c + zymst_c
SQLS: 2.0 frdp_c + h_c + nadph_c --> nadp_c + 

The reactions listed above can be used as targets for co-factor swapping to increase the theoretical yield of progesterone. From the 

In [472]:
model.genes.YGR192C  


Gene identifier,YGR192C
Name,TDH3
Memory address,0x07f5c15b956d0
Functional,False
In 1 reaction(s),GAPD


In [473]:
model.reactions.ALCD2x_copy2

Reaction identifier,ALCD2x_copy2
Name,Alcohol dehydrogenase (ethanol)
Memory address,0x07f5c15f71d00
Stoichiometry,etoh_c + nad_c <=> acald_c + h_c + nadh_c Ethanol + Nicotinamide adenine dinucleotide <=> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YDL168W
Lower bound,-999999.0
Upper bound,999999.0


In [474]:
model.metabolites.nad_c

Metabolite identifier,nad_c
Name,Nicotinamide adenine dinucleotide
Memory address,0x07f5c1b4037c0
Formula,C21H26N7O14P2
Compartment,c
In 48 reaction(s),"GAPD, PGCD, LNS14DMx, C3STDH2, GLUSx, ALCD25xi, GLUDxi, ALCD23xi, ALCD24xi, G3PD1ir, NMNAT, XYLTD_D, C22STDSx, R02216, NABTNO, SBTD_D2, NADN, PPND, SBTD_L, C3STDH1, G5SD2, LCADi, NADS1, FMNRx,..."


In [475]:
model.genes.YGR192C

Gene identifier,YGR192C
Name,TDH3
Memory address,0x07f5c15b956d0
Functional,False
In 1 reaction(s),GAPD


In [476]:
model.reactions.GAPD.genes

frozenset({<Gene YGR192C at 0x7f5c15b956d0>,
           <Gene YJL052W at 0x7f5c15b95790>,
           <Gene YJR009C at 0x7f5c15b957f0>})

In [477]:
model.genes.get_by_id("YJL052W")

Gene identifier,YJL052W
Name,TDH1
Memory address,0x07f5c15b95790
Functional,False
In 1 reaction(s),GAPD


In [478]:


# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

# Knock out of GAPD genes
KOs = ["YJL052W","YJR009C","YGR192C"]
model_coswap = model.copy()
for KO in KOs:
    model_coswap.genes.get_by_id(KO).knock_out()
    print(f"{KO} were knocked out")

# Adding NADP dependent GAPD reaction and NADP dependent alcohol dehydrogenase:
with open("data/co-factor_swap_reactions2.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        r = Reaction(line[0])
        r.name = line[2]
        if line[3] != "":
            r.subsystem = line[3]
        r.lower_bound = float(line[4])
        r.upper_bound = float(line[5])
        if line[1] != "":
            r.gpr = GPR.from_string(line[1])
        for i in range(int(len(line[6:])/2)):
            metaboliteID = line[6+i*2]
            if metaboliteID != "":
                metaboliteID = model_coswap.metabolites.get_by_id(metaboliteID)
                bin = float(line[7+i*2])
                r.add_metabolites({metaboliteID:bin})
        print(f'Adding reaction {r} | enzyme {r.gpr}')
        model_coswap.add_reactions([r])

# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')
# Add demand reaction for progesterone
model_coswap.add_boundary(model_coswap.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model_coswap.reactions.DM_progesterone_c.name}: {model_coswap.reactions.DM_progesterone_c.reaction}')

biomass = [model.optimize(),model_coswap.optimize()]

print(f'{model.id}: µ = {biomass[0].objective_value}')
print(f'{model_coswap.id}_coswap: µ = {biomass[1].objective_value}')

# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Model: {model.id} Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')
objective_reaction = model_coswap.reactions.R02216
model_coswap.objective = objective_reaction
print(f'Model: {model_coswap.id}_coswap Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')


pp = [model.optimize(),model_coswap.optimize()]

print(f'{model.id}: progesterone flux = {pp[0].objective_value}')
print(f'{model_coswap.id}: progesterone flux = {pp[1].objective_value}')


pp[0]


Loading models/iMM904_progesterone.xml
YJL052W were knocked out
YJR009C were knocked out
YGR192C were knocked out


FileNotFoundError: [Errno 2] No such file or directory: 'data/co-factor_swap_reactions2.csv'

In [ ]:
(biomass[1].objective_value / biomass[0].objective_value - 1)*100

2.0385317962620553

In [ ]:
for i, v in biomass[0].items():
    print('index: ', i, 'value: ', v)

AttributeError: 'Solution' object has no attribute 'items'

In [ ]:
biomass[0].index()

AttributeError: 'Solution' object has no attribute 'index'

In [ ]:
# for index,flux in biomass[0].fluxes:
#     print(index,flux)

# for x in biomass[0].fluxes.__dict__.items():
#     print(x[1].index())

type(biomass[0].fluxes)

pandas.core.series.Series

In [ ]:
type(biomass[0].to_frame())

pandas.core.frame.DataFrame

In [ ]:
test = pp[0].to_frame().fluxes.to_frame()
# print(test)
# for x in test.fluxes:
#     print(x)
for key, value in zip(test.index,test.fluxes):
    if key == "EX_glc__D_e":
        print(key,"\t",value)

EX_glc__D_e 	 -10.0


In [ ]:
r = model.reactions.get_by_id("R07215")
print(r.__dict__["_metabolites"])
print(r)
print("halli",model.metabolites.get_by_id("nad_c"))
modified_r = dict()
model_3betaHSD.genes.HSD3B.knock_out()
for i, metabolite in enumerate(r.metabolites):
    print(i)
    # print()
    r.sub_reaction()
    print(metabolite)
    print("jkjwlk",r.metabolites)
    if str(metabolite) in ("nad_c","nadh_c","nadp_c","nadph_c"):
        print(metabolite)
        print(model.metabolites.get_by_id(str(metabolite)))
        if str(metabolite) == "nad_c":
            print("hej")
            # model.metabolites.get_by_id(str(metabolite)) = 
            r.metabolites.metabolite = "nadh_c"
        elif str(metabolite) == "nadh_c":
            r.metabolites.metabolite = "nad_c"
        elif str(metabolite) == "nadp_c":
            print(r.metabolites.nadp_c)
            r.metabolites.nadp_c = model.metabolites.nadph_c
        elif str(metabolite) == "nadph_c":
            print(list(r.__dict__["_metabolites"])[i])
            print("hej",list(r.metabolites)[i])
            r.metabolites.nadph_c = model.metabolites.nadp
        else:
            modified_r[metabolite] = 
print(r)


{<Metabolite lathosterol_c at 0x7f5c264bc400>: -1.0, <Metabolite nadph_c at 0x7f5c246cf0d0>: -1.0, <Metabolite o2_c at 0x7f5c246d8a90>: -1.0, <Metabolite h_c at 0x7f5c2473f640>: -1.0, <Metabolite dehydrocholesterol_c at 0x7f5c2488c9d0>: 1.0, <Metabolite nadp_c at 0x7f5c246cfee0>: 1.0, <Metabolite h2o_c at 0x7f5c24737ee0>: 2.0}
R07215: h_c + lathosterol_c + nadph_c + o2_c --> dehydrocholesterol_c + 2.0 h2o_c + nadp_c
halli nad_c
0


AttributeError: 'Reaction' object has no attribute 'sub_reaction'

In [ ]:
# Loading model
infilename = 'models/iMM904.xml'
print(f"Loading {infilename}")
model = read_sbml_model(infilename)

Loading models/iMM904.xml


In [ ]:
# Add all new metabolites from metabolites.csv
new_metabolites = dict()
with open("data/metabolites.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        m = Metabolite(
            line[0],
            formula=line[1],
            name=line[2],
            compartment=line[3])
        print(f'Adding metabolite {m.name} (id: {m})')
        new_metabolites[line[0]] = m

Adding metabolite cholesterol (id: cholesterol_c)
Adding metabolite 7-dehydrocholesterol (id: dehydrocholesterol_c)
Adding metabolite pregnenolone (id: pregnenolone_c)
Adding metabolite progesterone (id: progesterone_c)
Adding metabolite 5alpha-Cholesta-7_24-dien-3beta-ol (id: cholesta724dien3betaol_c)
Adding metabolite 5alpha-Cholest-8-en-3beta-ol (id: cholesta8en3betaol_c)
Adding metabolite lathosterol (id: lathosterol_c)
Adding metabolite 4-methylpentanal (id: methylpentanal_c)


In [ ]:
# Make co-factor swap with DHCR7 all new reactions from reactions.csv
with open("data/co-factor_swap_reactions.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        r = Reaction(line[0])
        r.name = line[2]
        if line[3] != "":
            r.subsystem = line[3]
        r.lower_bound = float(line[4])
        r.upper_bound = float(line[5])
        if line[1] != "":
            r.gpr = GPR.from_string(line[1])
        for i in range(int(len(line[6:])/2)):
            metaboliteID = line[6+i*2]
            if metaboliteID != "":
                if metaboliteID in model.metabolites:
                    metaboliteID = model.metabolites.get_by_id(metaboliteID)
                else:
                    metaboliteID = new_metabolites[metaboliteID]
                bin = float(line[7+i*2])
                r.add_metabolites({metaboliteID:bin})
        print(f'Adding reaction {r} | enzyme {r.gpr}')
        model.add_reactions([r])

model.reactions.CHLSTI.gpr = GPR.from_string("EBP")
print(f'Adding gene annotation (EBP) to reaction: {model.reactions.CHLSTI}')

model.add_boundary(model.metabolites.get_by_id("methylpentanal_c"), type="sink")
print(f'Adding reaction {model.reactions.SK_methylpentanal_c.name}: {model.reactions.SK_methylpentanal_c.reaction}')

Adding reaction R07498: h_c + nadh_c + zymst_c <=> cholesta8en3betaol_c + nad_c | enzyme DHCR24
Adding reaction R05703: cholesta724dien3betaol_c + h_c + nadh_c <=> lathosterol_c + nad_c | enzyme DHCR24
Adding reaction R01456: dehydrocholesterol_c + h_c + nadh_c --> cholesterol_c + nad_c | enzyme DHCR7
Adding reaction ECYP11A1: cholesterol_c + 6.0 h_c + 6.0 nadh_c + 3.0 o2_c --> 4.0 h2o_c + methylpentanal_c + 6.0 nad_c + pregnenolone_c | enzyme CYP11A1
Adding reaction R02216: nadp_c + pregnenolone_c <=> h_c + nadph_c + progesterone_c | enzyme HSD3B
Adding reaction R03353: cholesta8en3betaol_c --> lathosterol_c | enzyme YMR202W
Adding reaction R07215: h_c + lathosterol_c + nadph_c + o2_c --> dehydrocholesterol_c + 2.0 h2o_c + nadp_c | enzyme YLR056W
Adding reaction R04804: zymst_c --> cholesta724dien3betaol_c | enzyme YMR202W
Adding gene annotation (EBP) to reaction: CHLSTI: amet_c + o2_c + zymst_c --> ahcys_c + ergtetrol_c + 2.0 h2o_c + h_c
Adding reaction 4-methylpentanal sink: methylp

In [ ]:
# Saving new model
outfilename = "models/iMM904_progesterone_coswap.xml"
model.id = outfilename.split("/")[-1].split(".")[0]
print(f"Saving to {outfilename}")
write_sbml_model(model, outfilename)

Saving to models/iMM904_progesterone_coswap.xml


In [ ]:
# Loading models
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model_progesterone = read_sbml_model(infilename)
infilename = 'models/iMM904_progesterone_coswap.xml'
print(f"Loading {infilename}")
model_progesterone_coswap = read_sbml_model(infilename)

Loading models/iMM904_progesterone.xml
Loading models/iMM904_progesterone_coswap.xml


In [ ]:
# Add demand reaction for progesterone
model_progesterone.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model_progesterone.reactions.DM_progesterone_c.name}: {model_progesterone.reactions.DM_progesterone_c.reaction}')
model_progesterone_coswap.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model_progesterone_coswap.reactions.DM_progesterone_c.name}: {model_progesterone_coswap.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 
Adding reaction progesterone demand: progesterone_c --> 


In [ ]:
print(f'{model_progesterone.id}: µ = {model_progesterone.optimize().objective_value}')
print(f'{model_progesterone_coswap.id}: µ = {model_progesterone_coswap.optimize().objective_value}')

iMM904_progesterone: µ = 0.2878657037040161
iMM904_progesterone_coswap: µ = 0.28786570370401676


In [ ]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model_progesterone.reactions.R02216
model_progesterone.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')
objective_reaction = model_progesterone_coswap.reactions.R02216
model_progesterone_coswap.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c
Objective set to maximize R02216: nadp_c + pregnenolone_c <=> h_c + nadph_c + progesterone_c


In [ ]:
print(f'{model_progesterone.id}: progesterone flux = {model_progesterone.optimize().objective_value}')
print(f'{model_progesterone_coswap.id}: progesterone flux = {model_progesterone_coswap.optimize().objective_value}')

iMM904_progesterone: progesterone flux = 0.14285714285714285
iMM904_progesterone_coswap: progesterone flux = 0.14285714285714288


In [ ]:
print(model_progesterone.genes.DHCR7.reactions)
print(model_progesterone_coswap.genes.DHCR7.reactions)

frozenset({<Reaction R01456 at 0x7f5c22aeea00>})
frozenset({<Reaction R01456 at 0x7f5c21c2ac40>})


In [ ]:
model_progesterone.reactions.R01456

Reaction identifier,R01456
Name,Cholesterol:NAD+ delta7-oxidoreductase
Memory address,0x07f5c22aeea00
Stoichiometry,dehydrocholesterol_c + h_c + nadph_c --> cholesterol_c + nadp_c 7-dehydrocholesterol + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> cholesterol + Nicotinamide adenine dinucleotide phosphate
GPR,DHCR7
Lower bound,0.0
Upper bound,999999.0


In [ ]:
model_progesterone_coswap.reactions.GAPD

Reaction identifier,GAPD
Name,Glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x07f5c22352c10
Stoichiometry,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate <=> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YJL052W or YJR009C or YGR192C
Lower bound,-999999.0
Upper bound,999999.0
